# Data Cleaning
The purpose of this notebook is to perform rapid data cleaning to prepare the final dataset for analysis and modeling. Its focus is orimarily on quick visualizaion, summaries, and statistics, as opposed to a completed deliverable product. To keep dataset manipulation separate from analysis and modeling, all changes to the datast will be made in this notebook.

For this sales predictive model, the target/predicted variable will be the opportunity stage which indicates whether an opportunity was won or lost. The goal of the project is to identify which features are most predictive of an opportunity being won or lost, *then* building a predictive model based on those features.

In [34]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# supress warnings about setting value on copy of slice
pd.options.mode.chained_assignment = None

# matplotlib and seaborn for visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# manipulate viz sizes
from IPython.core.pylabtools import figsize

# imputing missing values
from sklearn.impute import SimpleImputer

# machine learning models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

# model evauation
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, mean_absolute_error

# helper functions
import itertools

pd.set_option('display.max_columns', None)

In [35]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/data.csv')
data.head()

,Select Type of Sale,Type,Opportunity Name,Unit Type,Type of Purchase / Trial,Close Date,Lead Source,Opportunity Owner,Stage,Lost to Competitor,Competitor Incumbent,Total Opportunity Amount,Amount,Source of Funding,Created Date,Product Name,Quantity,Sales Price,Total Price,Product Date,Product: Month,Product Description,List Price,Product Code,Active Product,Account Name,Opportunity ID,Opportunity Score,Opportunity Quantity,Opportunity Record Type,Definitive Hospital ID,Definitive Hospital Name,Account ID,Medicare Provider Number,Number of Discharges,Total Acute Days,Payor Mix Medicare,Net Patient Revenue,HAI C.Diff Observed Cases 2015,HAI C.Diff Observed Cases 2016,HAI C.Diff Observed Cases 2017,HAI C.Diff Observed Cases 2018,HAI C.Diff Observed Cases 2019,HAI MRSA Observed Cases 2015,HAI MRSA Observed Cases 2016,HAI MRSA Observed Cases 2017,HAI MRSA Observed Cases 2018,HAI MRSA Observed Cases 2019,HAI SSI Abdominal Observed Cases 2018,HAI SSI Colon Observed Cases 2018,"HCAHPS % Reporting ""Always Clean"" 2015","HCAHPS % Reporting ""Always Clean"" 2016","HCAHPS % Reporting ""Always Clean"" 2017","HCAHPS % Reporting ""Always Clean"" 2018","HCAHPS % Reporting ""Always Clean"" 2019",Medicare VBP Adjustment 2017,Medicare VBP Adjustment 2018,Medicare VBP Adjustment 2019,Medicare VBP Adjustment 2020,Medicare VBP Adjustment 2021,Number of Staffed Beds,Account Owner,Firm Type,Hospital Type,Billing Street,Billing City,Billing State/Province,Billing Zip/Postal Code,Network,Network Parent,HAI C.Diff SIR 2015,HAI C.Diff SIR 2016,HAI C.Diff SIR 2017,HAI C.Diff SIR 2018,HAI C.Diff SIR 2019,HAI MRSA SIR 2015,HAI MRSA SIR 2016,HAI MRSA SIR 2017,HAI MRSA SIR 2018,HAI MRSA SIR 2019,HAI SSI Abdominal SIR 2018,HAI SSI Colon SIR 2018,Medicare HAC Reduction 2017,Medicare HAC Reduction 2018,Medicare HAC Reduction 2019,Medicare HAC Reduction 2020,Medicare HAC Reduction 2021,2017 HAC Score,2018 HAC Score,2019 HAC Score,2020 HAC Score,2021 HAC Score,Number of Medicare Discharges,Average Length of Stay,Average Daily Census,Account Type,Ownership,Geographic Classification,Employees,Medicare Certified Beds,Number of Operating Rooms,Number of ER Visits,Total Surgeries,Total Revenues,Bed Utilization Rate,Financial Data Date,Vertical Market,Sub Vertical,EVS Provider,GPO Affiliations
0,Product Sale,New Client,Blackburn Royal East Lancashir - Purchase New ...,Purchase New,NaN,2002-03-29,NaN,Jamie Blair,Engage,NaN,NaN,77000.0,77000.0,NaN,2021-02-04,Distributor LSMUV6-FT - International,1.0,77000.0,77000.0,2002-03-29,2002-03-01,Distributor LSMUV6-FT - International,90000.0,Distributor LSMUV6-FT - International,1,Blackburn Royal East Lancashire NHS Trust,0064R000019zhes,5.0,1.0,Product,NaN,NaN,0014R00002yQk8I,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.000,NaN,NaN,0.000,0.000,0.000,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vendor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Product Sale,Existing Client,North Shore Medical Center Sal - Purchase New ...,Purchase New,Capital Purchase,2021-11-04,NaN,Deb Wood,Budget,NaN,NaN,90300.0,90300.0,NaN,2021-02-05,LSMUV6-FT (unit # 1),1.0,105300.0,105300.0,2021-11-04,2021-11-01,LSMUV6-FT (unit # 1),143500.0,LSMUV6-FT,1,North Shore Medical Center Salem Hospital - Sa...,0064R000019zlau,34.0,2.0,Product,1935.0,North Shore Medical Center - Salem Hospital,001A000000R6aI4,220035,15287.0,70790.0,0.446,4.147521e+08,92.0,70.0,70.0,14.0,8,3.0,2.0,2.0,1.0,2,0.0,4.0,0.0,71.0,71.0,72.0,73,1.008514,1.007078,1.007078,1.007617,1.009992,304.0,Deb Wood,Hospital,Short Term Acute Care Hospital,81 Highland Ave,Salem,Massachusetts,1970.0,Mass General Brigham (FKA Partners HealthCare),Mass General Brigham (FKA Partners HealthCare),1.098,1.034,1.034,0.368,0.31,0.697,0.435,0.435,0.265,0.495,0.000,1.014,Yes,No,No,No,No,7.15,-0.2474,-0.2474,-0.3787,-0.0883,6739.0,4.6,182.2,Customer,Voluntary Nonprof

In [36]:
data.shape

(6064, 110)

110 Features x 6064 Samples

In [37]:
data.describe()

,Total Opportunity Amount,Amount,Quantity,Sales Price,Total Price,List Price,Active Product,Opportunity Score,Opportunity Quantity,Definitive Hospital ID,Number of Discharges,Total Acute Days,Payor Mix Medicare,Net Patient Revenue,HAI C.Diff Observed Cases 2015,HAI C.Diff Observed Cases 2016,HAI C.Diff Observed Cases 2017,HAI C.Diff Observed Cases 2018,HAI MRSA Observed Cases 2015,HAI MRSA Observed Cases 2016,HAI MRSA Observed Cases 2017,HAI MRSA Observed Cases 2018,HAI SSI Abdominal Observed Cases 2018,HAI SSI Colon Observed Cases 2018,"HCAHPS % Reporting ""Always Clean"" 2015","HCAHPS % Reporting ""Always Clean"" 2016","HCAHPS % Reporting ""Always Clean"" 2017","HCAHPS % Reporting ""Always Clean"" 2018",Medicare VBP Adjustment 2017,Medicare VBP Adjustment 2018,Medicare VBP Adjustment 2019,Medicare VBP Adjustment 2020,Medicare VBP Adjustment 2021,Number of Staffed Beds,Billing Zip/Postal Code,HAI C.Diff SIR 2015,HAI C.Diff SIR 2016,HAI C.Diff SIR 2017,HAI C.Diff SIR 2018,HAI MRSA SIR 2015,HAI MRSA SIR 2016,HAI MRSA SIR 2017,HAI MRSA SIR 2018,HAI SSI Abdominal SIR 2018,HAI SSI Colon SIR 2018,2017 HAC Score,2018 HAC Score,2019 HAC Score,2020 HAC Score,2021 HAC Score,Number of Medicare Discharges,Average Length of Stay,Average Daily Census,Employees,Medicare Certified Beds,Number of Operating Rooms,Number of ER Visits,Total Surgeries,Total Revenues,Bed Utilization Rate
count,6.064000e+03,4.365000e+03,2075.000000,2075.000000,4.365000e+03,2075.000000,6064.000000,1144.000000,5489.000000,5.229000e+03,4426.000000,4.426000e+03,4426.000000,4.426000e+03,5262.000000,5270.000000,5303.000000,5082.000000,5262.000000,5270.000000,5303.000000,5082.000000,5082.000000,5082.000000,5262.000000,5270.000000,5303.000000,5082.000000,3234.000000,3214.000000,3229.000000,3131.000000,3155.000000,5162.000000,5160.000000,5262.000000,5270.000000,5303.000000,5082.000000,5262.000000,5270.000000,5303.000000,5082.000000,5082.000000,5082.000000,5270.000000,3359.000000,3377.000000,3317.000000,3300.000000,4426.000000,4426.000000,4426.000000,4304.000000,5766.000000,5766.000000,5.765000e+03,5765.000000,5.765000e+03,5765.000000
mean,1.298403e+05,1.657568e+05,2.992622,40473.291041,1.288991e+05,50057.346863,0.336247,46.884615,4.300807,7.366938e+04,15093.072300,7.737992e+04,0.331791,5.486033e+08,33.766819,33.150285,37.096172,28.574183,2.768909,2.822011,3.252876,3.212121,0.937229,3.190476,47.659745,50.955271,52.556666,61.410862,1.000648,1.000506,1.000496,1.000794,1.000753,293.607129,53143.887984,0.622075,0.623730,0.642916,0.526365,0.467012,0.453327,0.477655,0.486087,0.338648,0.512994,3.645895,0.080762,0.081235,0.041014,0.013684,4310.271351,7.961591,201.800203,2252.373141,306.189559,9.066250,4.327849e+04,3242.381787,4.239206e+08,0.406325
std,2.259332e+05,2.409586e+05,14.156934,49088.746340,2.080826e+05,57465.043682,0.472463,39.404966,16.225256,1.959470e+05,20217.727724,1.101286e+05,0.163586,8.520997e+08,51.165840,49.044391,56.076787,43.629229,5.231951,5.363447,6.386134,5.659208,2.001229,5.391751,34.509886,33.412049,32.651881,26.819354,0.006529,0.005979,0.005972,0.005787,0.006903,417.198394,29505.587469,0.590332,0.581392,0.576065,0.468333,0.701519,0.650156,0.658438,0.633937,0.699655,0.626081,3.080424,0.542563,0.543360,0.520291,0.496508,6076.402995,41.239480,291.121567,3343.902805,456.150254,15.927899,7.123667e+04,5528.713068,8.268975e+08,0.295689
min,-6.200000e+04,-6.181700e+04,0.080000,-35000.000000,-1.250000e+05,-25000.000000,0.000000,1.000000,0.000000,1.000000e+00,0.000000,0.000000e+00,0.000000,-8.260666e+08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.985068,0.985107,0.985107,0.986202,0.980278,0.000000,936.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.720800,-1.720800,-1.589100,-1.725400,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000e+00,0.000000,-6.015949e+07,0.000000
25%,9.000000e+03,8.000000e+04,1.000000,640

In [38]:
data.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6064 entries, 0 to 6063
Data columns (total 110 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Select Type of Sale                     2804 non-null   object 
 1   Type                                    5826 non-null   object 
 2   Opportunity Name                        6064 non-null   object 
 3   Unit Type                               3971 non-null   object 
 4   Type of Purchase / Trial                1499 non-null   object 
 5   Close Date                              6064 non-null   object 
 6   Lead Source                             1529 non-null   object 
 7   Opportunity Owner                       6064 non-null   object 
 8   Stage                                   6064 non-null   object 
 9   Lost to Competitor                      267 non-null    object 
 10  Competitor Incumbent                    406 non-null    obj

Several columns currently encoded as object types will need to be cast to integers or floats:

* **HAI C.Diff Observed Cases 2019**
* **HAI MRSA Observed Cases 2019**
* **HAI C.Diff SIR 2019**
* **HAI MRSA SIR 2019**
* **HCAHPS % Reporting "Always Clean" 2019**
* **Medicare VBP Adjustment 2021**

Additionally, the **Defininitive Hospital ID** and **Billing Zip/Postal Code** features are read as floats because they are numeric variables. However, an object type would be more appropriate as it is a categorical variable.

## Convert Column Types

In [41]:
data = data.replace({'Not Available': np.nan})

# objects to floats
for col in list(data.columns):
  if ('HAI C.Diff Observed Cases 2019' in col or
      'HAI MRSA Observed Cases 2019' in col or
      'HAI C.Diff SIR 2019' in col or 
      'HAI MRSA SIR 2019' in col or
      'HCAHPS % Reporting "Always Clean" 2019' in col or
      'Medicare VBP Adjustment 2021' in col):
    data[col] = data[col].astype(float)

# floats to ojects
for col in list(data.columns):
  if ('Definitive Hospital ID' in col or
      'Billing Zip/Postal Code' in col):
    data[col] = data[col].astype(object)

data.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6064 entries, 0 to 6063
Data columns (total 110 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Select Type of Sale                     2804 non-null   object 
 1   Type                                    5826 non-null   object 
 2   Opportunity Name                        6064 non-null   object 
 3   Unit Type                               3971 non-null   object 
 4   Type of Purchase / Trial                1499 non-null   object 
 5   Close Date                              6064 non-null   object 
 6   Lead Source                             1529 non-null   object 
 7   Opportunity Owner                       6064 non-null   object 
 8   Stage                                   6064 non-null   object 
 9   Lost to Competitor                      267 non-null    object 
 10  Competitor Incumbent                    406 non-null    obj

## Decimal Percentages to Integers

Now that object types have been converted to more appropriate float types, specific type inconsistencies within columns of the same data type can be addressed. For example, there is a combination of both decimals and integers to represent percentages in the **HCAHPS % Reporting "Always Clean"** features for 2015 and 2016. Since most values are already in integer form, the percentages represented as decimals  will need be converted to their integer-equivalent.

In [58]:
# lambda expressions to convert decimal percentags to floats
data['HCAHPS % Reporting "Always Clean" 2015'] = data['HCAHPS % Reporting "Always Clean" 2015'].apply(lambda x: x * 100 if x < 1 else x)
data['HCAHPS % Reporting "Always Clean" 2016'] = data['HCAHPS % Reporting "Always Clean" 2016'].apply(lambda x: x * 100 if x < 1 else x)

## Grouping Categorical Variables

The Ownership and EVS Provider features are categorial variables with 14 and 13 unique values respectively, which can each be grouped into three subgroups to reduce the dimensions of the columns.


In [61]:
evs_unique = data['EVS Provider'].nunique()
owner_unique = data['Ownership'].nunique()
print('Unique EVS Providers:', evs_unique)
print('Unique Owenrships:', owner_unique)

Unique EVS Providers: 14
Unique Owenrships: 13


**EVS Providers**  are grouped into three categories: 
* In House
* Sodexo
* Commercial

**Ownership** values are also reduced to three general categoies:
* Government
* Nonprofit
* Proprietary

In [63]:
# replace null EVS providers with In House
data['EVS Provider'].fillna('In House', inplace=True)

# group commercian EVS providers
data['EVS Provider'].replace(['ABM', 'Aramark', 'Compass', 'Crothall', 'HHS', 'Other Commercial', 'Vanguard', 'Xanitos', 'Self-Op', 'In House'], 'Commercial', inplace=True)

# group Soedexo/International EVS providers
data['EVS Provider'].replace('International - Sodexo', 'Sodexo', inplace=True)

# group all goverment ownerships
data.loc[data.Ownership.str.contains('Governmental', na=False), 'Ownership'] = 'Government'

# group all nonprofit ownerships
data.loc[data.Ownership.str.contains('Nonprofit', na=False), 'Ownership'] = 'Nonprofit'

# group all proprietary ownerships
data.loc[data.Ownership.str.contains('Proprietary', na=False), 'Ownership'] = 'Proprietary'